In [1]:
from bs4 import BeautifulSoup
import requests
import re
import json

In [2]:
def get_html(url):
    response = requests.get(url)
    return response.text

In [3]:
tvs_url = 'https://rozetka.com.ua/all-tv/c80037/'

In [4]:
tv_soup = BeautifulSoup(get_html(tvs_url), 'lxml')

In [11]:
tv_title_divs = tv_soup.find_all('div', class_ = 'g-i-tile-i-title clearfix') 

tv_titles = []
for div in tv_title_divs:
    a = div.find('a')
    tv_titles.append((a.get("href"), a.text.strip()))
    
tv_titles[:5]

[('https://rozetka.com.ua/lg_43lk6200pld/p39024464/',
  'Телевизор LG 43LK6200PLD Оплата частями 12 мес. или Сертификат на 1500грн. в подарок!'),
 ('https://rozetka.com.ua/hisense_h43a6100_/p40090880/',
  'Телевизор Hisense H43A6100 Smart TV, 4K, 43"'),
 ('https://rozetka.com.ua/samsung_ue58nu7100uxua/p54812544/',
  'Телевизор Samsung UE58NU7100UXUA или Оплата частями на 24 мес.!'),
 ('https://rozetka.com.ua/samsung_ue49n5500auxua/p46780456/',
  'Телевизор Samsung UE49N5500AUXUA Оплата частями на 12 мес.!'),
 ('https://rozetka.com.ua/hisense_h55a6100/p40130128/',
  'Телевизор Hisense H55A6100 Smart TV, 4K, 55"')]

In [16]:
page_items = tv_soup.find_all('li', class_ = 'paginator-catalog-l-i pos-fix')

max_page_num = [int(item.get('id').replace('page', '')) for item in page_items][-1]

max_page_num

26

In [22]:
from collections import namedtuple

CatalogPage = namedtuple('CatalogPage', 'max_page_num items')

def get_items(catalog_url, page_num):
    page_url = catalog_url if page_num == 0 else catalog_url + 'page={}/'.format(page_num + 1)
    
    soup = BeautifulSoup(get_html(tvs_url), 'lxml')
    
    title_divs = tv_soup.find_all('div', class_ = 'g-i-tile-i-title clearfix') 

    titles = []
    for div in title_divs:
        a = div.find('a')
        titles.append((a.get("href"), a.text.strip()))
        
    page_items = tv_soup.find_all('li', class_ = 'paginator-catalog-l-i pos-fix')

    max_page_num = [int(item.get('id').replace('page', '')) for item in page_items][-1]    
    
    return CatalogPage(max_page_num, titles)

In [26]:
page_0 = get_items(tvs_url, 0)

In [27]:
len(page_1.items)

32

In [24]:
page_1 = get_items(tvs_url, 1)

In [25]:
len(page_1.items)

32

In [ ]:
<a href="https://rozetka.com.ua/philips_43pus6503_12/p39097504/comments/" class="novisited nav-tabs-link">
Отзывы<span class="nav-tabs-i-comments">91</span></a>